# Load and analyze data

In [1]:
import numpy as np
import pandas as pd


In [101]:
train_df = pd.read_csv("../data/train.csv", index_col="ID")
test_df = pd.read_csv("../data/test.csv", index_col="ID")
y = train_df["target"]
train_df.drop(["target"], axis=1, inplace=True)

 # Normalize data

In [94]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_norm = scaler.fit_transform(train_df)
test_norm = scaler.transform(test_df)

# apply pca

In [122]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)
reduced_train = pca.fit_transform(train_df)
reduced_test = pca.transform(test_df)

In [123]:
reduced_train.shape

(4459, 1221)

# Ridge loss

In [124]:
from sklearn.linear_model import Ridge
ridge = Ridge(random_state=17)
scores = cross_val_score(ridge, reduced_train, y, n_jobs = -1, verbose=1,  scoring="neg_mean_squared_log_error")
print(np.mean(np.sqrt(-scores)))

/home/snake/PycharmProjects/kaggle_competitions/.venv/lib/python3.5/site-packages/sklearn/metrics/regression.py:314: RuntimeWarning: invalid value encountered in log
  return mean_squared_error(np.log(y_true + 1), np.log(y_pred + 1),
/home/snake/PycharmProjects/kaggle_competitions/.venv/lib/python3.5/site-packages/sklearn/metrics/regression.py:314: RuntimeWarning: invalid value encountered in log
  return mean_squared_error(np.log(y_true + 1), np.log(y_pred + 1),
/home/snake/PycharmProjects/kaggle_competitions/.venv/lib/python3.5/site-packages/sklearn/metrics/regression.py:314: RuntimeWarning: invalid value encountered in log
  return mean_squared_error(np.log(y_true + 1), np.log(y_pred + 1),


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.5/site-packages/ipykernel_launcher.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.5/runpy.py in _run_code(code=<

# SVM CV

In [119]:
%%time
svm_clf = SVR(C=c, epsilon=e,cache_size=10000, verbose=True)
scores = cross_val_score(svm_clf, reduced_train, y, n_jobs = -1, verbose=1,  scoring="neg_mean_squared_log_error")
print(np.mean(np.sqrt(-scores)))

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   12.9s finished


1.7576713977612872
CPU times: user 192 ms, sys: 472 ms, total: 664 ms
Wall time: 13.8 s


In [118]:
np.mean(np.sqrt(-scores))

1.7576704987912395

In [129]:
from sklearn.svm import SVR
svm_clf_pca = SVR(C=1.0, epsilon=0.2,cache_size=10000, verbose=True)
svm_clf_pca.fit(reduced_train, y)

[LibSVM]

SVR(C=1.0, cache_size=10000, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=True)

In [131]:
pca_preds = svm_clf_pca.predict(reduced_test)

# SVM solution

In [128]:
%%time
from sklearn.svm import SVR
svm_clf_final = SVR(C=1.0, epsilon=0.2,cache_size=10000, verbose=True)
svm_clf_final.fit(train_norm, y)

[LibSVM]CPU times: user 2min 18s, sys: 216 ms, total: 2min 18s
Wall time: 2min 19s


In [96]:
%%time
svm_preds = svm_clf_final.predict(test_norm)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

# Combining solutions

In [134]:
final_preds = (svm_preds+pca_preds)/2.0
def prepare_score_file(scores):
    test_item_id = test_df.index
    sub_df = pd.DataFrame({"ID":test_item_id})
    sub_df["target"] = scores
    sub_df.to_csv("/tmp/santander.csv", index=False)
prepare_score_file(final_preds)